# Diagonalizing coordinate space solutions in 3D

In this notebook you will apply a 2nd-derivative operator and a potential as matrices in coordinate space to represent the Schroedinger equation. By diagonalizing it you'll find its eigenvalues (the energy spectrum) and eigenvectors (wave functions).

What is different here from previous notebooks is that we'll do it in three dimensions. This requires very few changes if we solve for the radial wave function $u_{E\ell}(r) = r R_{E\ell}(r)$. The radial Schrodinger equation for an energy eigenstate is:

$$
 \hat H |\psi\rangle = E |\psi\rangle\quad \doteq\quad  \Bigl[ -\frac{\hbar^2}{2m}\frac{d^2}{dr^2} + \frac{\hbar^2\ell(\ell+1)}{2mr^2} +  V(r)\Bigr] u_{El}(r) = E u_{El}(r) ,
$$

where the full wavefunction is $\psi(\mathbf{x}) = R_{E\ell}(r) Y^m_{\ell}(\theta,\phi)$. (Be careful not to confuse the mass $m$ with the quantum number $m$. The latter will not appear again in this notebook, however, because the energy $E$ and radial function $u_{El}$ depend on $\ell$ but not on this $m$.)

Standard imports plus seaborn (to make plots looks nicer).

In [1]:
import numpy as np
import scipy.linalg as la

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("darkgrid"); sns.set_context("talk")


We use the now-familiar  `second_derivative_matrix` function below so that it returns a matrix that implements an approximate second derivative when applied to a vector made up of a function evaluated at the mesh points. Now we apply it to functions of $r$ rather than functions of $x$, but that is just labeling; the 2nd derivative has the same form. The numpy `diag` and `ones` functions are used to create matrices with 1's on particular diagonals, as in this $5\times 5$ example: 

$$ \frac{1}{(\Delta r)^2}\,\left( 
    \begin{array}{ccccc}
    -2 & 1 & 0 & 0 & 0 \\
    1 & -2 & 1 & 0 & 0 \\
    0 & 1 & -2 & 1 & 0 \\
    0 & 0 &1 & -2 & 1 \\
    0 & 0 & 0 & 1 & -2
    \end{array}
   \right) 
   \left(\begin{array}{c}
         f_1 \\ f_2 \\ f_3 \\ f_4 \\ f_5
         \end{array}
   \right) 
   %\overset{?}{=}
   =
   \left(\begin{array}{c} -2f_1 + f_2
          \\ f_1 - 2 f_2 + f_3  \\ f_2 - 2 f_3 + f_4 \\ f_3 - 2 f_4 + f_5 \\ f_4 - 2 f_5
         \end{array}
   \right) 
 $$  
 

In [2]:
def second_derivative_matrix(N, Delta_r):
    """
    Return an N x N matrix for 2nd derivative of a vector equally spaced by delta_x
    """
    M_temp = np.diag(np.ones(N-1), +1) + np.diag(np.ones(N-1), -1) \
              - 2 * np.diag(np.ones(N), 0)

    return M_temp / (Delta_r**2)

## Setting up the second derivative

**Choose values for `N_pts`, `x_min`, and `x_max`** 

In [24]:
N_pts = 4001   # For accuracy this should be large; over 1000 
r_min = 0.     # For three-dimensions, r starts at zero
r_max = 5.     # How large to make this depends on the potential
Delta_r = (r_max - r_min) / (N_pts - 1)
r_mesh = np.linspace(r_min, r_max, N_pts)  # create the grid ("mesh") of x points

Set up the derivative matrices for the specified mesh.

In [25]:
second_deriv = second_derivative_matrix(N_pts, Delta_r)

## Setting up potentials and Hamiltonian matrix

The Hamiltonian matrix is 

$$
 \hat H \doteq  -\frac{\hbar^2}{2m}\frac{d^2}{dr^2} + \frac{\hbar^2\ell(\ell+1)}{2mr^2} +  V(r) ,
$$

which we'll implement as a sum of matrices. We'll choose units initially so that $\hbar = 1$ and $m = 1$, but these can be set to any desired values.

In [26]:
def V_SHO_matrix(r_mesh):
    """
    Harmonic oscillator potential matrix (defined as a diagonal matrix)
    """
    k = 1/2       # k is chosen so that hbar*omega = 1 
    V_diag = k * r_mesh**2 / 2  # diagonal matrix elements
    N = len(r_mesh)  # number of x points
    
    return V_diag * np.diag(np.ones(N), 0) 

In [27]:
def Three_D_spherical_box(r_mesh, ell, a, beta, mass=1, hbar=1):
    """
    Potential in a box of radius a and depth hbar^2 beta^2 / (2 m a^2).
    """
    V0 = hbar**2 * beta**2 / (2 * mass * a**2)
    Lfact = hbar**2 * ell * (ell + 1) / (2 * mass)
    V_diag = np.array([V0 if r > a else 0 for r in r_mesh])
    V_diag = V_diag + np.array([Lfact / r**2 if r > 0.001 else 0 for r in r_mesh])
    N = len(r_mesh)  # number of r points
    
    return V_diag * np.diag(np.ones(N), 0)     

**Put together the Hamiltonian matrix**

In [28]:
hbar = 1
mass = 1

ell = 0   # pick the value of orbital angular momentum l
a = 1     # radius of the spherical box
beta = 10 # parameter setting the height of the potential wall
V_spherical = Three_D_spherical_box(r_mesh, ell, a, beta, mass=1, hbar=1)

Hamiltonian = -hbar**2/(2*mass) * second_deriv + V_spherical

# Diagonalize using numpy functions
eigvals, eigvecs = np.linalg.eigh(Hamiltonian)

gs = eigvals[0]

print(f' *** Energy eigenvalues for l = {ell} ***')
print(' n       E       ka      ka/pi')
for n, eig in enumerate(eigvals[0:10]):
    print(f' {n}  {eig:8.4f}  {np.sqrt(eig*2*mass*a**2/hbar**2):7.3f} ', \
          f' {np.sqrt(eig*2*mass*a**2/hbar**2)/np.pi:6.3f}')


 *** Energy eigenvalues for l = 0 ***
 n       E       ka      ka/pi
 0    4.0541    2.848    0.906
 1   16.0728    5.670    1.805
 2   35.3629    8.410    2.677
 3   50.2976   10.030    3.193
 4   51.1827   10.118    3.221
 5   52.6291   10.260    3.266
 6   54.5781   10.448    3.326
 7   56.9121   10.669    3.396
 8   59.5045   10.909    3.472
 9   62.4796   11.179    3.558


In [29]:
def get_eigvals(ell, beta, num_eigs=5):
    """
    Get the first num_eigs eigenvalues for specified ell and beta 
    """
    hbar = 1
    mass = 1
    a = 1

    V_spherical = Three_D_spherical_box(r_mesh, ell, a, beta, 
                                        mass=mass, hbar=hbar)
    Hamiltonian = -hbar**2/(2*mass) * second_deriv + V_spherical
    # Diagonalize using numpy functions
    eigvals, eigvecs = np.linalg.eigh(Hamiltonian)

    return eigvals[0:num_eigs]

In [32]:
betas = np.array([4, 10, 25, 100, 1000])
num_eigs = 5

for ell in [0, 1]:
    print(f' orbital angular momentum l = {ell}')
    print(f' beta   {num_eigs} * [eigenvalue   sqrt(eig*2ma^2/hbar^2)]')
    for beta in betas:
        print(f' {beta:4.0f} ', end =" ")
        eigs = get_eigvals(ell, beta, num_eigs)
        for i in range(num_eigs):
            print(f'{eigs[i]:7.3f} ', \
                  f'{np.sqrt(eigs[i]*2*mass*a**2/hbar**2):5.3f}', 
                  end =" ")
        print(' ')    
    print(' ')

 orbital angular momentum l = 0
 beta   5 * [eigenvalue   sqrt(eig*2ma^2/hbar^2)]
    4    3.053  2.471   8.264  4.065   9.011  4.245  10.138  4.503  11.632  4.823  
   10    4.054  2.848  16.073  5.670  35.363  8.410  50.298  10.030  51.183  10.118  
   25    4.545  3.015  18.170  6.028  40.841  9.038  72.494  12.041 113.015  15.034  
  100    4.819  3.105  19.278  6.209  43.374  9.314  77.106  12.418 120.473  15.522  
 1000    4.905  3.132  19.619  6.264  44.143  9.396  78.475  12.528 122.616  15.660  
 
 orbital angular momentum l = 1
 beta   5 * [eigenvalue   sqrt(eig*2ma^2/hbar^2)]
    4    6.022  3.470   8.449  4.111   9.421  4.341  10.912  4.672  12.857  5.071  
   10    8.293  4.073  24.186  6.955  46.268  9.620  50.451  10.045  51.451  10.144  
   25    9.319  4.317  27.524  7.419  54.766  10.466  90.964  13.488 135.966  16.490  
  100    9.884  4.446  29.214  7.644  58.201  10.789  96.848  13.917 145.152  17.038  
 1000   10.059  4.485  29.732  7.711  59.235  10.884  98.571  

In [ ]:
norm = 1 / np.sqrt(Delta_r)

wf_0 = norm * eigvecs[:,0]
wf_1 = norm * eigvecs[:,1]
wf_2 = norm * eigvecs[:,2]

print(f' test normalization: {Delta_r * wf_0 @ wf_0}')


In [ ]:
fig = plt.figure(figsize=(16,6))

ax1 = fig.add_subplot(1,2,1)
ax1.set_xlabel(r'$r$')
ax1.set_ylabel(r'$\psi_n(r)$')

r_max = 3 * a
ax1.set_xlim(0, r_max)
#ax1.set_ylim(-1., 3)

ax1.plot(r_mesh, wf_0, color='red', label=r'$n=0$')
ax1.plot(r_mesh, wf_1, color='blue', label=r'$n=1$')
ax1.plot(r_mesh, wf_2, color='green', label=r'$n=2$')

ax1.set_title(fr'Lowest energy eigenstates for $\ell$ = {ell}')

ax1.legend();


In [ ]:
hbar = 1
mass = 1

ell = 1   # pick the value of orbital angular momentum l
a = 1     # radius of the spherical box
beta = 10 # parameter setting the height of the potential wall
V_spherical = Three_D_spherical_box(r_mesh, ell, a, beta, mass=1, hbar=1)

Hamiltonian = -hbar**2/(2*mass) * second_deriv + V_spherical

# Diagonalizing using numpy functions
eigvals, eigvecs = np.linalg.eigh(Hamiltonian)

E_gs = eigvals[0]  # ground state energy
print(f' *** Energy eigenvalues for l = {ell} ***')
print(' n       E       ka    ')
for n, eig in enumerate(eigvals[0:10]):
    print(f' {n}  {eig:8.4f}  {np.sqrt(eig*2*mass*a**2/hbar**2):7.3f} ')
    #, \
    #      f' {np.sqrt(eig*2*mass*a**2/hbar**2)/np.pi:6.3f}')



In [ ]:
norm = 1 / np.sqrt(Delta_r)

wf_0 = norm * eigvecs[:,0]
wf_1 = norm * eigvecs[:,1]
wf_2 = norm * eigvecs[:,2]

print(f' test normalization: {Delta_r * wf_0 @ wf_0}')


In [ ]:
fig = plt.figure(figsize=(16,6))

ax1 = fig.add_subplot(1,2,1)
ax1.set_xlabel(r'$r$')
ax1.set_ylabel(r'$\psi_n(r)$')

r_max = 3 * a
ax1.set_xlim(0, r_max)
#ax1.set_ylim(-1., 3)

ax1.plot(r_mesh, wf_0, color='red', label=r'$n=0$')
ax1.plot(r_mesh, wf_1, color='blue', label=r'$n=1$')
ax1.plot(r_mesh, wf_2, color='green', label=r'$n=2$')

ax1.set_title(fr'Lowest energy eigenstates for $\ell$ = {ell}')

ax1.legend();
